# Description:

    In this part of project, I performed the classification of CIFAR-10 dataset (which consists of 60000 32x32 colour images in 10 classes, with 6000 images per class) using simple neural network. I performed the similar task in 'Classifier-Interpretability' but by using Convolution Neural Network, but here i performed it by using simple Neural network.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim


# Downloading Data

In [2]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
#preparing data to use for training and testing
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

# Preparing the Model:

In [4]:
model = nn.Sequential(nn.Linear(3072,1536),
                      nn.ReLU(),
                      nn.Linear(1536, 768),
                      nn.ReLU(),
                      nn.Linear(768,128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

epochs = 100
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
#         print(loss.item())
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: 2.3030110917737723
Training loss: 2.297347385560155
Training loss: 2.2892305164995705
Training loss: 2.271051549545639
Training loss: 2.2197014642188617
Training loss: 2.1349926174754073
Training loss: 2.0484177758321738
Training loss: 1.9746740791193969
Training loss: 1.916922664550869
Training loss: 1.8643530003554987
Training loss: 1.8159899781731998
Training loss: 1.771785099183202
Training loss: 1.729953258543673
Training loss: 1.6899296139817104
Training loss: 1.6523250002995171
Training loss: 1.6194021226195119
Training loss: 1.5893274983176795
Training loss: 1.5606901641087154
Training loss: 1.53184601748386
Training loss: 1.5032461117905425
Training loss: 1.4755301884068248
Training loss: 1.4488104112312923
Training loss: 1.420985269272114
Training loss: 1.3932349427276864
Training loss: 1.3675845323316276
Training loss: 1.341305897135259
Training loss: 1.3147959509469054
Training loss: 1.2882611035081126
Training loss: 1.2630675206403903
Training loss: 1.237933

In [5]:
images_t, labels_t = next(iter(testloader))


In [6]:
images_t.shape

torch.Size([64, 3, 32, 32])

In [7]:
images_t=images_t.view(images_t.shape[0], -1)

In [8]:
images_t.shape

torch.Size([64, 3072])

The goal of validation is to measure the model's performance on data that isn't part of the training set. Performance here is up to the developer to define though. Typically this is just accuracy, the percentage of classes the network predicted correctly. Other options are precision and recall and top-5 error rate. We'll focus on accuracy here. First I'll do a forward pass with one batch from the test set.

In [9]:
# images_t, labels_t = next(iter(testloader))
# images_t=images_t.view(images_t.shape[0], -1)
# Get the class probabilities
ps = torch.exp(model(images_t))
# # Make sure the shape is appropriate, we should get 10 class probabilities for 64 examples
print(ps.shape)

torch.Size([64, 10])


With the probabilities, we can get the most likely class using the ps.topk method. This returns the  𝑘  highest values. Since we just want the most likely class, we can use ps.topk(1). This returns a tuple of the top- 𝑘  values and the top- 𝑘  indices. If the highest value is the fifth element, we'll get back 4 as the index.

In [10]:
top_p, top_class = ps.topk(1, dim=1)
# Look at the most likely classes for the first 10 examples
print(top_class[:10,:],len(top_class[:,:]))

tensor([[0],
        [0],
        [0],
        [5],
        [6],
        [6],
        [6],
        [6],
        [0],
        [8]]) 64


Equals is comparing the one element in each row of top_class with each element in labels

In [11]:
equals = top_class == labels_t.view(*top_class.shape)

In [12]:
accuracy = torch.mean(equals.type(torch.FloatTensor))
print(f'Accuracy: {accuracy.item()*100}%')

Accuracy: 42.1875%
